In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
import seaborn as sns

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_pickle(f'../workdata/miro/part_4.pkl')
df = df.groupby(['diag', 'db']).head(5)
df.groupby(['diag', 'db']).size()

diag  db                  
AF    WFDB_CPSC2018           5
      WFDB_CPSC2018_2         5
      WFDB_ChapmanShaoxing    5
      WFDB_Ga                 5
      WFDB_PTB                3
      WFDB_PTBXL              5
SR    WFDB_CPSC2018           5
      WFDB_ChapmanShaoxing    5
      WFDB_Ga                 5
      WFDB_Ningbo             5
      WFDB_PTB                4
      WFDB_PTBXL              5
dtype: int64

In [27]:
leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
sns.set(rc={'figure.figsize':(11.7,8.27)})
df2 = pd.DataFrame(columns=['diag', 'data'])

if not os.path.exists(f'../workdata/aioros/SR/'):
    os.makedirs(f'../workdata/aioros/SR/')
    
if not os.path.exists(f'../workdata/aioros/AF/'):
    os.makedirs(f'../workdata/aioros/AF/')

for i, row in df.iterrows():
    try:
        ecg_signal = row['data'][-2] # V5
        _, rpeaks = nk.ecg_peaks(ecg_signal, sampling_rate=100)
        ecg_r_peaks = rpeaks['ECG_R_Peaks']
        ecg_r_peaks = list(map(lambda x: x - 20, ecg_r_peaks))
        middle_signal = int(len(ecg_r_peaks)/2)
        boundies = ecg_r_peaks[middle_signal:middle_signal+2]
        diag = row["diag"]
        new_row = {
            'diag': diag,
            'data': row['data'][:, boundies[0]:boundies[1]]
        }

        if not os.path.exists(f'../workdata/aioros/{diag}/sample_{i}/'):
            os.makedirs(f'../workdata/aioros/{diag}/sample_{i}/')

        plot = sns.lineplot(
            x=np.arange(0, len(ecg_signal), 1, dtype=int),
            y=ecg_signal)
        plot.axvline(boundies[0], color='red')
        plot.axvline(boundies[1], color='red')
        fig = plot.get_figure()
        fig.set_size_inches(25,15)
        fig.savefig(f"../workdata/aioros/{diag}/sample_{i}/sample_v5.png", facecolor='white', transparent=False, dpi=300) 
        fig.clf()

        for j, lead in enumerate(leads):
            plot = sns.lineplot(
                x=np.arange(0, boundies[1]-boundies[0], 1, dtype=int), 
                y=new_row['data'][j])
            fig = plot.get_figure()
            fig.savefig(f"../workdata/aioros/{diag}/sample_{i}/lead_{lead}.png", facecolor='white', transparent=False, dpi=300) 
            fig.clf()
            df2 = df2.append(new_row, ignore_index=True)
    except Exception as e:
        print(e)

<Figure size 1800x1080 with 0 Axes>